# Parte B: Preprocesado de datos

## **1.** Cargar resultados de la parte anterior.

📝 Carga del archivo `df_unico.csv` que generamos en la Parte A: obtención de datos. 


In [2]:
import pandas as pd
path = '/Users/Usuario PC/OneDrive/Documentos/Esp. Bioinformática/TIAD/df_unico'

# Leemos el archivo df_unico.csv y lo guardamos en un DataFrame
df = pd.read_csv('df_unico.csv', sep=',', comment='#')


C:\Users\Usuario PC\AppData\Local\Temp\ipykernel_17516\902138006.py:5: DtypeWarning: Columns (17,22,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_unico.csv', sep=',', comment='#')


In [3]:
print(df.head())

print(f"-----------------------------------")
print(f"El DataFrame tiene {len(df)} filas y {len(df.columns)} columnas.")

     PATIENT_ID     SEX   AGE METASTASIS   OS_STATUS  OS_MONTHS  \
0  TCGA-A2-A0T2  Female  66.0         M1  1:DECEASED       7.89   
1  TCGA-A2-A04P  Female  36.0         M0  1:DECEASED      17.97   
2  TCGA-A1-A0SK  Female  54.0         M0  1:DECEASED      31.77   
3  TCGA-A2-A0CM  Female  40.0         M0  1:DECEASED      24.77   
4  TCGA-AR-A1AR  Female  50.0         M0  1:DECEASED      17.18   

         SAMPLE_ID ER_STATUS PR_STATUS HER2_STATUS  ... SLC2A11   GRIP2  \
0  TCGA-A2-A0T2-01  Negative  Negative    Negative  ...  2.1991  0.1544   
1  TCGA-A2-A04P-01  Negative  Negative    Negative  ... -0.6395 -0.2865   
2  TCGA-A1-A0SK-01  Negative  Negative    Negative  ... -0.6395 -0.2865   
3  TCGA-A2-A0CM-01  Negative  Negative    Negative  ... -0.6395 -0.2865   
4  TCGA-AR-A1AR-01  Negative  Negative    Negative  ... -0.6395 -0.2865   

   GPLD1   RAB8A   RXFP2 PIK3IP1 SLC39A6  SNRPD2    AQP7    CTSC  
0 -0.363 -0.1779  1.1753 -1.6969 -0.9826  0.1062 -0.5218 -0.5842  
1 -0.363  1.

## **2.** Limpieza y numerizado.

Como se aplicarán un tipo de técnicas de análisis específico sobre los datos, los mismos se deben preprocesar antes de usarlos.

### Limpieza

En nuestro DataFrame tenemos filas de datos que tienen campos con valores nulos (NaN). Se identifican cuales son.

In [4]:
# Mostrar cantidad de valores NaN por columna
nan_counts = df.isna().sum()
print(nan_counts)

# Mostrar cantidad de filas del conjunto de datos
print(f"-----------------------------------")
print(f"El DataFrame tiene {len(df)} filas.")

PATIENT_ID      0
SEX             7
AGE             7
METASTASIS     40
OS_STATUS       7
             ... 
PIK3IP1       299
SLC39A6       299
SNRPD2        299
AQP7          299
CTSC          299
Length: 17302, dtype: int64
-----------------------------------
El DataFrame tiene 825 filas.


En este caso nos interesa deshacernos de aquellas filas donde el atributo `METASTASIS` sea `NaN`

📝 **a)** Eliminar las filas donde el atributo `METASTASIS` sea `NaN`.

In [5]:
# Eliminar filas con valores donde el atributo 'METASTASIS' tiene valor nulos

df = df.dropna(subset=['METASTASIS']) 

In [6]:
# Mostrar cantidad de valores NaN por columna
nan_counts = df.isna().sum()
print(nan_counts)

print(f"-----------------------------------")
# La cantidad de filas deberia haber reducido en 40
print(f"El DataFrame tiene {len(df)} filas .")

PATIENT_ID      0
SEX             0
AGE             0
METASTASIS      0
OS_STATUS       0
             ... 
PIK3IP1       266
SLC39A6       266
SNRPD2        266
AQP7          266
CTSC          266
Length: 17302, dtype: int64
-----------------------------------
El DataFrame tiene 785 filas .


Las filas que tienen valores nulos en `METASTASIS` coinciden con aquellas donde otros atributos como `SEX`, `OS_STATUS` tampoco estaban definidos. Es una buena situación, pero para lograrlo tuvimos que sacrificar varias filas que si tienen información en otros campos.

Dependiendo de la situación hay que valorar que merece mas la pena:
- Eliminar las filas con valores `NaN`: si el atributo con valor `NaN` no es muy importante capaz no vale la pena perder los demás datos de la fila.
- Eliminar columna con atributos `NaN`: esto tiene la contra de que se pierde el atributo para todas las filas. No obstante, es una buena opción si la columna tiene en su mayor parte valores `NaN`.
- Suplantar los `NaN` con otros valores: Se puede asignar un valor a todos los campos que tengan `NaN` pero esto se tiene que hacer de forma dedicada a cada campo y con cuidado, por que podría equivaler a inventarse datos.
- Dejar los valores `NaN`: hay algoritmos que lo toleran.

Estos son algunos posibles cursos de acción para el preprocesamiento, el que hacer en cada caso depende del criterio propio, los datos de los que dispongamos y las técnicas que queramos aplicar.

### Numerizado

La variable `df` contiene atributos tanto categóricos como numéricos. Nos interesa que todos los atributos sean numéricos por lo que tendremos que transformar aquellos que no lo sean.

In [7]:
# Mostrar los nombres y tipos de todas las columnas que NO sean de tipo numerico
print(df.select_dtypes(exclude='number').dtypes)

PATIENT_ID              object
SEX                     object
METASTASIS              object
OS_STATUS               object
SAMPLE_ID               object
ER_STATUS               object
PR_STATUS               object
HER2_STATUS             object
TUMOR_STAGE             object
TUMOR_T1_CODED          object
NODES                   object
NODE_CODED              object
METASTASIS_CODED        object
CONVERTED_STAGE         object
SURVIVAL_DATA_FORM      object
PAM50_SUBTYPE           object
RPPA_CLUSTER            object
CANCER_TYPE_DETAILED    object
ONCOTREE_CODE           object
CANCER_TYPE             object
SAMPLE_TYPE             object
SOMATIC_STATUS          object
subtype                 object
dtype: object


En esta ocasión, al realizar la carga de los archivos de datos en un DataFrame, todas las columnas con valores de tipo numérico se cargaron correctamente. No obstante, todas las demás columnas fueron catalogadas como object. Esto es una buena catalogación para *SAMPLE_ID* y para *PATIENT_ID*. Pero no para todos los demás campos.

A continuación se cambiará el tipo de columna de object a category en todas las varibles que correspondan.

In [8]:
# Cambiar tipo de columna SEX
df['SEX'] = df['SEX'].astype('category')

In [9]:
print(df.select_dtypes(exclude='number').dtypes)

PATIENT_ID                object
SEX                     category
METASTASIS                object
OS_STATUS                 object
SAMPLE_ID                 object
ER_STATUS                 object
PR_STATUS                 object
HER2_STATUS               object
TUMOR_STAGE               object
TUMOR_T1_CODED            object
NODES                     object
NODE_CODED                object
METASTASIS_CODED          object
CONVERTED_STAGE           object
SURVIVAL_DATA_FORM        object
PAM50_SUBTYPE             object
RPPA_CLUSTER              object
CANCER_TYPE_DETAILED      object
ONCOTREE_CODE             object
CANCER_TYPE               object
SAMPLE_TYPE               object
SOMATIC_STATUS            object
subtype                   object
dtype: object


📝 Corrección de todos los atributos que están clasificados como object a excepción de *PATIENT_ID* y *SAMPLE_ID*.

In [10]:

df['METASTASIS'] = df['METASTASIS'].astype('category')
df['OS_STATUS'] = df['OS_STATUS'].astype('category')
df['ER_STATUS'] = df['ER_STATUS'].astype('category')
df['PR_STATUS'] = df['PR_STATUS'].astype('category')
df['HER2_STATUS'] = df['HER2_STATUS'].astype('category')
df['TUMOR_STAGE'] = df['TUMOR_STAGE'].astype('category')
df['TUMOR_T1_CODED'] = df['TUMOR_T1_CODED'].astype('category')
df['TUMOR_T1_CODED'] = df['TUMOR_T1_CODED'].astype('category')
df['NODES'] = df['NODES'].astype('category')
df['NODE_CODED'] = df['NODE_CODED'].astype('category')
df['METASTASIS_CODED'] = df['METASTASIS_CODED'].astype('category')
df['CONVERTED_STAGE'] = df['CONVERTED_STAGE'].astype('category')
df['SURVIVAL_DATA_FORM'] = df['SURVIVAL_DATA_FORM'].astype('category')
df['PAM50_SUBTYPE'] = df['PAM50_SUBTYPE'].astype('category')
df['RPPA_CLUSTER'] = df['RPPA_CLUSTER'].astype('category')
df['CANCER_TYPE_DETAILED'] = df['CANCER_TYPE_DETAILED'].astype('category')
df['ONCOTREE_CODE'] = df['ONCOTREE_CODE'].astype('category')
df['CANCER_TYPE'] = df['CANCER_TYPE'].astype('category')
df['SAMPLE_TYPE'] = df['SAMPLE_TYPE'].astype('category')
df['SOMATIC_STATUS'] = df['SOMATIC_STATUS'].astype('category')
df['TMB_NONSYNONYMOUS'] = df['TMB_NONSYNONYMOUS'].astype('category')
df['subtype'] = df['subtype'].astype('category')
# ------------------------------------------------------

In [11]:
print(df.select_dtypes(exclude='number').dtypes)

PATIENT_ID                object
SEX                     category
METASTASIS              category
OS_STATUS               category
SAMPLE_ID                 object
ER_STATUS               category
PR_STATUS               category
HER2_STATUS             category
TUMOR_STAGE             category
TUMOR_T1_CODED          category
NODES                   category
NODE_CODED              category
METASTASIS_CODED        category
CONVERTED_STAGE         category
SURVIVAL_DATA_FORM      category
PAM50_SUBTYPE           category
RPPA_CLUSTER            category
CANCER_TYPE_DETAILED    category
ONCOTREE_CODE           category
CANCER_TYPE             category
SAMPLE_TYPE             category
SOMATIC_STATUS          category
TMB_NONSYNONYMOUS       category
subtype                 category
dtype: object


#### **a)** Numerización de identificadores únicos

Los campos PATIENT_ID y SAMPLE_ID corresponden a identificadores únicos (no va a haber 2 filas con los mismos valores en PATIENT_ID y/o en SAMPLE_ID).

📝 Mapeo de los valores de las columnas PATIENT_ID y SAMPLE_ID a valores numéricos, mapeando cada código un número distinto.

In [12]:
import pandas as pd

# Se utiliza la funcion `factorize` de la libreria
# pandas para mapear una los valores de una columna a valores numéricos)
df['PATIENT_ID'] = pd.factorize(df['PATIENT_ID'])[0] + 1  
df['SAMPLE_ID'] = pd.factorize(df['SAMPLE_ID'])[0] + 1  

In [13]:
print('---------------------------------------------')
print(df['PATIENT_ID'].head())
print(f"La columa tiene {len(df['PATIENT_ID'])} filas.")
print('---------------------------------------------')
print(df['SAMPLE_ID'].head())
print(f"La columa tiene {len(df['SAMPLE_ID'])} filas.")
print('---------------------------------------------')

---------------------------------------------
0    1
1    2
2    3
3    4
4    5
Name: PATIENT_ID, dtype: int64
La columa tiene 785 filas.
---------------------------------------------
0    1
1    2
2    3
3    4
4    5
Name: SAMPLE_ID, dtype: int64
La columa tiene 785 filas.
---------------------------------------------


#### **b)**  Numerización de ordinales

El caso del atributo `TUMOR_STAGE` puede contener los valores: *T1*, *T2*, *T3* y *T4*. En este caso sería correcto catalogar `TUMOR_STAGE` como un atributo **categórico ordinal**.

In [14]:
# Mostrar lista de posibles valores de la columna TUMOR_STAGE
print(df['TUMOR_STAGE'].unique().tolist())

['T3', 'T2', 'T1', 'TX', 'T4']


Nota: El DataFrame tiene 2 filas donde `TUMOR_STAGE == 'TX'`. Esto significa que no se conoce en qué etapa se encuentra un tumor. No nos interesan estos casos por lo que simplemente los borraremos

In [15]:
import numpy as np

# Nos quedamos solo con las filas que son distintas a 'TX'
df = df[df['TUMOR_STAGE'] != 'TX']

In [16]:
print(df['TUMOR_STAGE'].unique().tolist())

['T3', 'T2', 'T1', 'T4']



📝 A continuación se mapea los valores de la columna *TUMOR_STAGE* a valores numéricos donde se vea reflejado el criterio de ordenación *T1 < T2 < T3 < T4*.

In [17]:
# Diccionario de mapeo
stage_mapping = {'T1': 1, 'T2': 2, 'T3': 3, 'T4': 4}

# Aplica el mapa a la columna TUMOR_STAGE 
df['TUMOR_STAGE'] = df['TUMOR_STAGE'].map(stage_mapping)  

In [18]:
# Mostrar primeros valores de la columna
print(df['TUMOR_STAGE'].head())
print(f"La columa tiene {len(df['TUMOR_STAGE'])} filas.")

0    3.0
1    2.0
2    2.0
3    2.0
4    1.0
Name: TUMOR_STAGE, dtype: float64
La columa tiene 783 filas.


In [19]:
# Volver a mostrar la lista de posibles valores de la columna TUMOR_STAGE
print(df['TUMOR_STAGE'].unique().tolist())

[3.0, 2.0, 1.0, 4.0]


📝 A continuación se mapea de la misma forma los valores del atributo 'METASTASIS'

In [20]:
# Numerizar 'METASTASIS'
stage_mapping = {'M0': 0, 'M1': 1, 'M2': 2, 'M3': 3, 'M4': 4} 
df['METASTASIS'] = df['METASTASIS'].map(stage_mapping)        
print(df['METASTASIS'].unique().tolist())

[1, 0]


#### **c)** Numerizacion de nominales

Hay otros atributos como `PAM50_SUBTYPE` que son **categóricos nominales**. Osea que no necesariamente existe un orden entre sus distintas categorías.

Para numerizar este tipo de casos se empleará la técnica *One-Hot Encoding*.

📝 Numerización de la columna categórica `PAM50_SUBTYPE` acorde a la técnica *One-Hot Encoding*.

In [21]:
# Mostrar lista de posibles valores de la columna PAM50_SUBTYPE
print(df['PAM50_SUBTYPE'].unique().tolist())

print('---------------------------------------------')
print(df['PAM50_SUBTYPE'].head())
print(f"El DataFrame tiene {len(df)} filas y {len(df.columns)} columnas.")

['Basal-like', 'HER2-enriched', 'Luminal A', 'Luminal B', 'Normal-like', nan]
---------------------------------------------
0    Basal-like
1    Basal-like
2    Basal-like
3    Basal-like
4    Basal-like
Name: PAM50_SUBTYPE, dtype: category
Categories (5, object): ['Basal-like', 'HER2-enriched', 'Luminal A', 'Luminal B', 'Normal-like']
El DataFrame tiene 783 filas y 17302 columnas.


In [22]:
import pandas as pd

# Numerización ONE-HOT de la columna 'PAM50_SUBTYPE' con el método 'get_dummies' de la librería pandas 
df = pd.get_dummies(df, columns=['PAM50_SUBTYPE']) 

In [23]:
print(df.head())
print(f"El DataFrame tiene {len(df)} filas y {len(df.columns)} columnas.")

   PATIENT_ID     SEX   AGE METASTASIS   OS_STATUS  OS_MONTHS  SAMPLE_ID  \
0           1  Female  66.0          1  1:DECEASED       7.89          1   
1           2  Female  36.0          0  1:DECEASED      17.97          2   
2           3  Female  54.0          0  1:DECEASED      31.77          3   
3           4  Female  40.0          0  1:DECEASED      24.77          4   
4           5  Female  50.0          0  1:DECEASED      17.18          5   

  ER_STATUS PR_STATUS HER2_STATUS  ...  PIK3IP1 SLC39A6  SNRPD2    AQP7  \
0  Negative  Negative    Negative  ...  -1.6969 -0.9826  0.1062 -0.5218   
1  Negative  Negative    Negative  ...  -2.5818 -0.9826  1.0638 -0.3011   
2  Negative  Negative    Negative  ...  -0.0933 -0.9826 -0.7018 -0.5218   
3  Negative  Negative    Negative  ...  -0.5590 -0.9826  0.7508 -0.5218   
4  Negative  Negative    Negative  ...  -0.3168 -0.9826  0.1956  2.1747   

     CTSC PAM50_SUBTYPE_Basal-like PAM50_SUBTYPE_HER2-enriched  \
0 -0.5842                 

**Nota:** De acuerdo a la versión de pandas que se esté usando las columnas derivadas de `PAM50_SUBTYPE` (ósea `[PAM50_SUBTYPE_Basal-like, PAM50_SUBTYPE_HER2-enriched, PAM50_SUBTYPE_Luminal A, PAM50_SUBTYPE_Luminal B, PAM50_SUBTYPE_Normal-like]`) no quedan con una codificación numérica `[0, 1]`, sino con una codificación booleana `[True, False]`. Es aceptable también.

📝 Numerización de la columna categórica `PE_STATUS` acorde a la técnica *One-Hot Encoding*.

In [24]:
# Mirar valores de 'ER_STATUS'
print(df['ER_STATUS'].unique().tolist())

['Negative', 'Positive', 'Performed but Not Available', 'Not Performed', 'Indeterminate']


In [25]:
# Numerizar 'ER_STATUS'
df = pd.get_dummies(df, columns=['ER_STATUS']) 

#### **d)** Numerizacion de binarios

Por último hay otros atributos como `OS_STATUS` que solo tienen 2 posibles valores `['1:DECEASED', '0:LIVING']`. En estos casos basta con hacer que el campo tome valor 1 para uno de estos dos valores y 0 para el otro.

📝 Numerización de el atributo binario `OS_STATUS`.

In [26]:
# Mostrar lista de posibles valores de la columna TUMOR_STAGE
print(df['OS_STATUS'].unique().tolist())

print('---------------------------------------------')
print(df['OS_STATUS'].head())
print(f"La columa tiene {len(df['OS_STATUS'])} filas.")

['1:DECEASED', '0:LIVING']
---------------------------------------------
0    1:DECEASED
1    1:DECEASED
2    1:DECEASED
3    1:DECEASED
4    1:DECEASED
Name: OS_STATUS, dtype: category
Categories (2, object): ['0:LIVING', '1:DECEASED']
La columa tiene 783 filas.


In [27]:
import numpy as np

# Numerización de el atributo OS_STATUS con la función 'where' de la librería de numpy
# (Nota: también se podría haber hecho con el mapeo de posibles valores como en los anteriores) 

df['OS_STATUS'] = np.where(df['OS_STATUS'] == '0:LIVING', 0, 1)

In [28]:
# Mostrar lista de posibles valores de la columna TUMOR_STAGE
print(df['OS_STATUS'].unique().tolist())

print('---------------------------------------------')
print(df['OS_STATUS'].head())
print(f"La columa tiene {len(df['OS_STATUS'])} filas.")

[1, 0]
---------------------------------------------
0    1
1    1
2    1
3    1
4    1
Name: OS_STATUS, dtype: int64
La columa tiene 783 filas.


📝 A continuación se numeriza la variable binaria 'SEX' 

In [29]:
# Numerizar 'SEX'
df['SEX'] = np.where(df['SEX'] == 'Female', 1, 0) 
print(df['SEX'].unique().tolist())

[1, 0]


📝 Finalmente se muestra el DataFrame con todas las columnas para poder observar si la numerización de las variables se realizó correctamente. 

In [30]:
# Mostrar primeros valores de cada columna
print(df.head())
print(f"La columa tiene {len(df)} filas.")

   PATIENT_ID  SEX   AGE METASTASIS  OS_STATUS  OS_MONTHS  SAMPLE_ID  \
0           1    1  66.0          1          1       7.89          1   
1           2    1  36.0          0          1      17.97          2   
2           3    1  54.0          0          1      31.77          3   
3           4    1  40.0          0          1      24.77          4   
4           5    1  50.0          0          1      17.18          5   

  PR_STATUS HER2_STATUS  TUMOR_STAGE  ... PAM50_SUBTYPE_Basal-like  \
0  Negative    Negative          3.0  ...                     True   
1  Negative    Negative          2.0  ...                     True   
2  Negative    Negative          2.0  ...                     True   
3  Negative    Negative          2.0  ...                     True   
4  Negative    Negative          1.0  ...                     True   

  PAM50_SUBTYPE_HER2-enriched PAM50_SUBTYPE_Luminal A PAM50_SUBTYPE_Luminal B  \
0                       False                   False            

## **3.** Almacenamiento de resultados.

📝 Una vez que tenemos los datos procesados procedemos a guardar el DataFrame en el formato csv. 


In [31]:
nombre = 'df_preprocesado.csv'
df.to_csv(nombre, index=False) 